Global setup

In [256]:
n=4
k=2
q=2^6-2^2+1
eta=1
gamma1=2^3
gamma2=(q-1)/4
R.<x> = ZZ[]

In [257]:
q

61

In [258]:
is_prime(q)

True

In [259]:
gamma2

15

In [260]:
for tau in range(5):
    print(ln(binomial(4, tau)).n()+tau)

0.000000000000000
2.38629436111989
3.79175946922805
4.38629436111989
4.00000000000000


In [261]:
tau = 2
beta = tau*eta
beta

2

Alice generates a signing key

In [194]:
A = matrix([[randrange(0,q)*x^3+randrange(0,q)*x^2+randrange(0,q)*x+randrange(0,q), randrange(0,q)*x^3+randrange(0,q)*x^2+randrange(0,q)*x+randrange(0,q)], 
            [randrange(0,q)*x^3+randrange(0,q)*x^2+randrange(0,q)*x+randrange(0,q), randrange(0,q)*x^3+randrange(0,q)*x^2+randrange(0,q)*x+randrange(0,q)]])
A

[ 41*x^3 + 50*x^2 + 32*x + 3 41*x^3 + 55*x^2 + 60*x + 15]
[ 50*x^3 + 10*x^2 + 7*x + 55        35*x^3 + 49*x^2 + 24]

In [195]:
s1 = matrix([[randrange(-eta,eta)*x^3+randrange(-eta,eta)*x^2+randrange(-eta,eta)*x+randrange(-eta,eta)], 
            [randrange(-eta,eta)*x^3+randrange(-eta,eta)*x^2+randrange(-eta,eta)*x+randrange(-eta,eta)]])
s1

[-x^3 - x^2 - x]
[      -x^3 - x]

In [196]:
s2 = matrix([[randrange(-eta,eta)*x^3+randrange(-eta,eta)*x^2+randrange(-eta,eta)*x+randrange(-eta,eta)], 
            [randrange(-eta,eta)*x^3+randrange(-eta,eta)*x^2+randrange(-eta,eta)*x+randrange(-eta,eta)]])
s2

[-x^3 - x^2 - x - 1]
[          -x^2 - x]

$s_1$ and $s_2$ are private

In [197]:
t = matrix(((A*s1 + s2) % (x^n+1) % q))
t

[ 27*x^3 - 14*x^2 + 5*x + 40]
[38*x^3 + 22*x^2 + 29*x + 41]

$A$ and $t$ are public

Alice signs the message

In [432]:
M = 'hi!'
M

'hi!'

In [433]:
y = matrix([[randrange(-(gamma1-1), gamma1-1)*x^3+randrange(-(gamma1-1), gamma1-1)*x^2+randrange(-(gamma1-1), gamma1-1)*x+randrange(-(gamma1-1), gamma1-1)], 
            [randrange(-(gamma1-1), gamma1-1)*x^3+randrange(-(gamma1-1), gamma1-1)*x^2+randrange(-(gamma1-1), gamma1-1)*x+randrange(-(gamma1-1), gamma1-1)]])
y

[3*x^3 + 3*x^2 + 3*x - 5]
[    x^3 - 5*x^2 - x + 2]

In [434]:
def high_bits(r, alpha):
    r = r % q
    r0 = Mod(r, alpha).lift_centered()
    if r-r0 == q-1:
        return 0
    else:
        return (r-r0)/alpha

def low_bits(r, alpha):
    r = r % q
    r0 = Mod(r, alpha).lift_centered()
    if r-r0 == q-1:
        return r0-1
    else:
        return r0

In [435]:
matrix(((A*y) % (x^n+1) % q))

[36*x^3 + 32*x^2 + 14*x + 24]
[11*x^3 + 51*x^2 + 52*x + 35]

In [436]:
w1 = matrix(((A*y) % (x^n+1) % q)).apply_map(lambda f: f.map_coefficients(lambda r: high_bits(r, 2*gamma2) ))
w1

[x^3 + x^2 + 1]
[            1]

In [437]:
print(w1)

[x^3 + x^2 + 1]
[            1]


In [438]:
from sage.cpython.string import str_to_bytes

In [439]:
str_to_bytes(M+str(w1))

b'hi![x^3 + x^2 + 1]\n[            1]'

In [440]:
import hashlib

In [441]:
c_hash = hashlib.shake_256(str_to_bytes(M+str(w1)))

In [442]:
def sample_in_ball(digest):  # digest should be twice the length of c
    c = [0] * 4
    for i in range(4-tau, 4):
        j = digest[i] % (i+1)
        s = digest[i+4] % 2
        c[i] = c[j]
        c[j] = (-1)^s
    return c
    

In [443]:
c = R(sample_in_ball(c_hash.digest(8)))
c

x^2 - x

In [444]:
z = matrix(((y + c*s1) % (x^n+1) % q))
z

[3*x^3 + 4*x^2 + 4*x - 5]
[             -4*x^2 + 1]

Is $\|\mathbf{z}\|_\infty \geq \gamma_1-\beta$?

In [445]:
gamma1-beta

6

If so, reject and pick a new $\mathbf{y}$

Is $\|\mathrm{LowBits}(\mathbf{Ay}-c\mathbf{s}_2, 2\gamma_2)\|_\infty \geq \gamma_2-\beta$?

In [446]:
matrix(((A*y - c*s2) % (x^n+1) % q)).apply_map(lambda f: f.map_coefficients(lambda r: low_bits(r, 2*gamma2) ))

[ 6*x^3 + 2*x^2 + 12*x - 6]
[11*x^3 - 11*x^2 - 9*x + 4]

In [447]:
gamma2-beta

13

If so, reject and pick a new $\mathbf{y}$

If not, $(\mathbf{z}, c)$ is the signature

In [458]:
(z, c)

(
[3*x^3 + 4*x^2 + 4*x - 5]         
[             -4*x^2 + 1], x^2 - x
)

Bob verifies the signature

In [459]:
w1prime = matrix(((A*z-c*t) % (x^n+1) % q)).apply_map(lambda f: f.map_coefficients(lambda r: high_bits(r, 2*gamma2) ))
w1prime

[x^3 + x^2 + 1]
[            1]

Is $\|\mathbf{z}\|_\infty < \gamma_1-\beta$?

In [450]:
z

[3*x^3 + 4*x^2 + 4*x - 5]
[             -4*x^2 + 1]

In [451]:
gamma1-beta

6

Yes

Is $c = H(M \| w_1')$?

In [462]:
cprime_hash = hashlib.shake_256(str_to_bytes(M+str(w1prime)))

In [466]:
cprime = R(sample_in_ball(cprime_hash.digest(8)))
c

x^2 - x

In [467]:
c == cprime

True